## Linkler ↓↓↓

-https://python-binance.readthedocs.io/en/latest/

-https://readthedocs.org/projects/python-binance/downloads/pdf/stable/

##  İmportlar

In [1]:
from binance.client import Client
import config
import time

## Client tanımlama

In [2]:
client = Client(config.API_KEY, config.API_SECRET)

## Basamak sayısını elde etme

In [3]:
def getPrecision(Ticker):
    while(True):
        try:
            info = client.futures_exchange_info()
            requestedFutures = [Ticker]
            ListOfPrecisions = {si['symbol']:si['quantityPrecision'] for si in info['symbols'] if si['symbol'] in requestedFutures}
            return ListOfPrecisions.get(Ticker)
        except Exception as e :
            print(f"{Ticker} getPrecision couldnt get")
            time.sleep(1)
getPrecision("BTCUSDT")

3

## Kaldıraç Değeri Değiştirme

In [ ]:
def ChangeLeverage(Symbol,Leverage):
    x = 0
    while(x < 5):
        try:
            client.futures_change_leverage(symbol=Symbol,leverage=Leverage,maxNotionalValue="1000000")
            break
        except Exception as e:
            print(f"{Symbol} change levereage error ! {e}")
            time.sleep(1)
            x = x + 1

ChangeLeverage("BTCUSDT",125)

## Marjin tipini değiştirme

In [ ]:
def changeMarginType(Symbol,stringMargin):
    x = 0
    while(x < 5):
        try:
            client.futures_change_margin_type(symbol=Symbol,marginType=stringMargin)
            break
        except Exception as e : 
            print(f"{Symbol} marginType error ! {e}")
            time.sleep(1)
            x = x + 1

changeMarginType("BTCUSDT","ISOLATED")

## Anlık Fiyat

In [4]:
def getCurrentPrice(Symbol):
    while(True):
        try:
            price = client.futures_symbol_ticker(symbol=Symbol)
            price = float(price.get("price"))
            return price
        except Exception as e :
            print(f"{Symbol} currentPrice couldnt get ! {e}")
            time.sleep(1)

print(getCurrentPrice("BTCUSDT"))

38922.0


## Vadeli Emirler

### Uzun market tipi pozisyon açmak

In [12]:
def openMarketLong(Symbol,Quantity):
    while(True):
        try:
            client.futures_create_order(symbol=Symbol, side="BUY", type="MARKET",quantity=Quantity,positionSide="LONG")
            break
        except Exception as e :
            print(f"{Symbol} open market long error ! {e}")
            time.sleep(1)

currentPrice = getCurrentPrice("BTCUSDT")
precision = getPrecision("BTCUSDT")
leverage = 15
qtyDollar = 14
quantity = round((qtyDollar/currentPrice*leverage),precision)

openMarketLong("BTCUSDT",quantity)


38997.50000


### Uzun market tipi pozisyonu kapatmak

In [25]:
def getPositionQuantityLong(Symbol):
    while(True):
        try:
            orders = client.futures_position_information(symbol=Symbol)
            order = orders[1]
            quantity = order.get("positionAmt")
            quantity= float(quantity)
            return quantity
        except Exception as e:
            print(f"{Symbol} get quantity error ! {e}")
            time.sleep(1)

print(getPositionQuantityLong("BTCUSDT"))


def closeMarketLong(Symbol,precision):
    while(True):
        try:
            quantity = round(getPositionQuantityLong(Symbol),precision)
            client.futures_create_order(symbol=Symbol, side="SELL", type="MARKET", quantity=quantity, positionSide="LONG")
            break
        except Exception as e :
            print(f"{Symbol} closeLong error ! {e}")
            time.sleep(1)

precision = getPrecision("BTCUSDT")
closeMarketLong("BTCUSDT",precision)

0.005


### Kısa market tipi pozisyon açmak

In [30]:
def openMarketShort(Symbol,Quantity):
    while(True):
        try:
            client.futures_create_order(symbol=Symbol, side="SELL", type="MARKET", quantity=Quantity, positionSide="SHoRT")
            break
        except Exception as e:
            print(f"{Symbol} open market short error ! {e}")
            time.sleep(1)

currentPrice = getCurrentPrice("BTCUSDT") # Anlik fiyat
precision = getPrecision("BTCUSDT") # Futures basamak sayisi
leverage= 15
qtyDollar = 10
quantity = round((qtyDollar/currentPrice*leverage),precision)

openMarketShort("BTCUSDT",quantity)

### Kısa market tipi pozisyon kapatmak

In [34]:
def getPositionQuantityShort(Symbol):
    while(True):
        try:
            orders = client.futures_position_information(symbol=Symbol)
            order = orders[::-1][0]
            quantity = order.get("positionAmt")
            quantity = float(quantity[1:])
            return quantity
        except Exception as e:
            print(f"{Symbol} get quantity error ! {e}")
            time.sleep(1)

print(getPositionQuantityShort("BTCUSDT"))


def closeMarketShort(Symbol,precision):
    while(True):
        try:
            quantity = round(getPositionQuantityShort(Symbol),precision)
            client.futures_create_order(symbol=Symbol, side="BUY", type="MARKET", quantity=quantity, positionSide="SHORT")
            break
        except Exception as e :
            print(f"{Symbol} closeShort error ! {e}")
            time.sleep(1)

precision = getPrecision("BTCUSDT")
closeMarketShort("BTCUSDT",precision)

0.004


### Uzun limit emri vermek

In [36]:
def openLimitLong(Symbol,Quantity,Price):
    while(True):
        try:
            client.futures_create_order(symbol=Symbol, side="BUY", type="LIMIT", quantity=Quantity, positionSide="LONG",price=Price,timeInForce="GTC")
            break
        except Exception as e :
            print(f"{Symbol} open limit long error ! {e}")
            time.sleep(1)

currentPrice = getCurrentPrice("BTCUSDT")
precision = getPrecision("BTCUSDT")
leverage = 15
qtyDollar = 10
quantity = round((qtyDollar/currentPrice*leverage),precision)

openLimitLong("BTCUSDT",quantity,"39940")

### Kısa limit emri vermek

In [37]:
def openLimitShort(Symbol,Quantity,Price):
    while(True):
        try:
            client.futures_create_order(symbol=Symbol, side="SELL", type="LIMIT", quantity=Quantity, positionSide="SHORT",price=Price,timeInForce="GTC")
            break
        except Exception as e :
            print(f"{Symbol} open limit short error ! {e}")
            time.sleep(1)

currentPrice = getCurrentPrice("BTCUSDT")
precision = getPrecision("BTCUSDT")
leverage = 15
qtyDollar = 10
quantity = round((qtyDollar/currentPrice*leverage),precision)

openLimitShort("BTCUSDT",quantity,"40810")

### Açık emri iptal etmek

In [44]:
# Get order ID
def getOpenOrderID(Symbol):
    while(True):
        try:
            openOrders = client.futures_get_open_orders(symbol = Symbol)
            orderID = openOrders[0]["orderId"]
            return orderID
        except Exception as e :
            print(f"get open order error ! {e}")


orderID = getOpenOrderID("BTCUSDT")
print(f"order ID: {orderID}")

def cancelOrderById(Symbol,orderID):
    while(True):
        try:
            client.futures_cancel_order(symbol=Symbol,orderId=orderID)
            break
        except Exception as e:
            print(f"cancel order error ! {e}") 
            time.sleep(1)


cancelOrderById("BTCUSDT",orderID)



order ID: 51549065630
